In [ ]:
%run _prepare.ipynb

# Analyse time

Time is the fourth dimension and an essential component of most datasets, therefore it is often helpful to analyse important features at different points of time.

The most common used charts for those are:

* line-charts: Visualize independent timelines 
* area-charts: Visualize aggregating timelines

## Continent-CO2 Timeline

### Using pandas plotting backend

In [ ]:
plot_df_pandas = countries.groupby(["year","continent"]).co2.sum().unstack("continent").reset_index()
plot_df_pandas.plot(x="year", figsize=(20,5))

### Using plotly

In [ ]:
plot_df_px = countries.groupby(["year","continent"]).co2.sum().reset_index()
px.line(plot_df_px, y="co2", x="year", color="continent")

### Using Altair

In [ ]:
alt.data_transformers.disable_max_rows()
countries["dt"] = pd.to_datetime(countries.year.astype(str) + "-01-01")

alt.Chart(countries, width=1009, height=250).mark_line().encode(
    x="year", 
    y=alt.Y(field='co2', aggregate='sum', type='quantitative'),
    color="continent"
)

# More Timelines with Altair

In [ ]:
raw_data_info.co2_per_capita

In [ ]:
alt.data_transformers.disable_max_rows()
countries["dt"] = pd.to_datetime(countries.year.astype(str) + "-01-01")
alt.Chart(countries, width=900, height=250).mark_line().encode(
    x="dt:T", 
    y=alt.Y(field='co2_per_capita', aggregate='mean', type='quantitative'),
    color="continent"
)

In [ ]:
alt.data_transformers.disable_max_rows()
countries["dt"] = pd.to_datetime(countries.year.astype(str) + "-01-01")
def plot_chart(Y = alt.Y(field='co2_per_capita', aggregate='mean', type='quantitative')):
    return alt.Chart(countries, width=900, height=250).mark_line().encode(
        x="dt:T", 
        y = Y,
        color="continent"
    )

plot_chart(
    alt.Y(field='co2_per_capita', aggregate='mean', type='quantitative')
) & plot_chart(
    alt.Y(field='co2', aggregate='sum', type='quantitative')
)

# 📝 Task: Make a similiar chart that shows the usage of the different co2 sources

* Hint: Altair requires the [long-data-format](https://de.wikipedia.org/wiki/Wide-Format_und_Long-Format). Use the `.melt()` command an your dataframe

In [ ]:
plot_df = countries[["country","continent", "year", *cols_co2_sources]].melt(["country","continent","year"])

In [ ]:
alt.data_transformers.disable_max_rows()
co2_sources_chart = alt.Chart(plot_df, width=900, height=250).mark_line().encode(
    x="year", 
    y=alt.Y(field='value', aggregate='sum', type='quantitative'),
    color="variable"
)

In [ ]:
co2_sources_chart

# Altair Interactivity

In [ ]:
import altair as alt
from vega_datasets import data


brush = alt.selection(type='interval', encodings=['x'])

upper = co2_sources_chart.encode(
    alt.X('year', scale=alt.Scale(domain=brush))
)

lower = co2_sources_chart.properties(
    height=60
).add_selection(brush)

upper & lower

And all of this is still combinable with altairs other super-powers. For example try:
`upper.mark_area() & upper.encode(row="continent") & lower`

In [ ]:
plot_df = countries[filter_most_recent].join(countries.groupby("country").agg({"co2": sum, "co2_per_capita": "mean"}).add_suffix("_historical"), on="country").query("co2 >1")

In [ ]:
scatter = alt.Chart(plot_df).mark_circle().encode(
    x="co2_per_capita",
    y="co2",
    color="continent",
    size="co2_historical",
    tooltip=["country", "co2", "co2_per_capita", "co2_historical",  "co2_per_capita_historical"]
).properties(
    width=1000,
    height=400
)
scatter

In [ ]:
selection = alt.selection_multi(fields=['country'], empty='none')

alt.data_transformers.disable_max_rows()
line_co2 = alt.Chart(countries, width=600, height=150).mark_line().encode(
    x="dt:T", 
    y="co2",
    color="country:N",
    tooltip = ["country", "co2"]
).transform_filter(selection)

dashboard = alt.hconcat(
    scatter.properties(width=400, height=450).add_selection( selection ),
    line_co2 & line_co2.encode(y="co2_per_capita"),
    line_co2.encode(y="total_ghg") & line_co2.encode(y="gdp")
)
dashboard.resolve_scale(color='independent').configure(autosize=alt.AutoSizeParams(resize=True))

# 📝 Task: Extend the dashboard to gain insights you will be interested in

Ideas:
* modify fields in the scatter plot
* add additional timelines
* add comparative charts

## Next level dashbaord

In [ ]:
selection = alt.selection_multi(fields=['country'], empty='none')
nearest = alt.selection(type='single', nearest=True, on='mouseover',fields=['dt'], empty='none')

alt.data_transformers.disable_max_rows()
line_co2 = alt.Chart(countries, width=600, height=150).mark_line().encode(
    x="dt:T", 
    y="co2",
    color="country:N",
    tooltip = ["country", "co2"]
).transform_filter(selection)

selectors = alt.Chart(countries).mark_point().encode(
    x='dt:T',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw a rule at the location of the selection
rules = alt.Chart(countries).mark_rule(color='gray').encode(
    x='dt:T',
).transform_filter(
    nearest
)

line = line_co2 +  rules

dashboard = alt.hconcat(
    scatter.properties(width=400, height=450).add_selection( selection ),
    line+ selectors & line.encode(y="co2_per_capita:Q"),
    line.encode(y="total_ghg:Q") & line.encode(y="gdp:Q")
)
dashboard.resolve_scale(color='independent').configure(autosize=alt.AutoSizeParams(resize=True))